# Regmap Parsing

In [1]:
import ipq_pynq_utils
import numpy as np

In [2]:
clk104 = ipq_pynq_utils.CLK104("clockfile.txt")

In [3]:
clk104.PLL2_FREQ = 2457.6 # Main Reference Clock
# clk104.PLL2_FREQ = 3000 # Main Reference Clock

print(f"ADC_REFCLK: {clk104.ADC_REFCLK.freq:.2f}")
print(f"DAC_REFCLK: {clk104.DAC_REFCLK.freq:.2f}")
print(f"RF_PLL_ADC_REF: {clk104.RF_PLL_ADC_REF.freq:.2f}")
print(f"RF_PLL_DAC_REF: {clk104.RF_PLL_DAC_REF.freq:.2f}")
print(f"SYSREF_FREQ: {clk104.SYSREF_FREQ:.2f}")

ADC_REFCLK: 245.76
DAC_REFCLK: 245.76
RF_PLL_ADC_REF: 245.76
RF_PLL_DAC_REF: 245.76
SYSREF_FREQ: 7.68


In [4]:
for i,branch in enumerate(clk104.lmk.clock_branches):
    print(i, branch.dclk_active, branch.sdclk_active)

0 True True
1 False True
2 True True
3 False False
4 True True
5 False True
6 False False


In [5]:
clk104.get_register_dump()

[16,
 512,
 774,
 1232,
 1371,
 1536,
 3153,
 3332,
 65546,
 65877,
 66133,
 66304,
 66594,
 66816,
 67312,
 67377,
 67594,
 67925,
 68181,
 68352,
 68642,
 68864,
 69360,
 69424,
 69642,
 69973,
 70229,
 70400,
 70690,
 70912,
 71408,
 71475,
 71690,
 72021,
 72277,
 72448,
 72738,
 72960,
 73457,
 73488,
 73736,
 74069,
 74325,
 74496,
 74786,
 75008,
 75504,
 75569,
 75786,
 76117,
 76373,
 76544,
 76834,
 77056,
 77552,
 77616,
 77834,
 78165,
 78421,
 78592,
 78882,
 79104,
 79601,
 79616,
 79872,
 80131,
 80385,
 80704,
 80896,
 81153,
 81411,
 81666,
 81923,
 82176,
 82432,
 82704,
 83199,
 83327,
 83515,
 83738,
 83974,
 84294,
 84486,
 84758,
 84992,
 85248,
 85696,
 85887,
 86019,
 86274,
 86528,
 86784,
 87048,
 87296,
 87560,
 87808,
 88189,
 88320,
 88704,
 89041,
 89120,
 89344,
 89600,
 89918,
 90112,
 90393,
 90692,
 90880,
 91136,
 91552,
 94634,
 94722,
 97301,
 97587,
 91648,
 91904,
 92352,
 92505,
 92704,
 92928,
 93184,
 93440,
 93723,
 94976,
 98816,
 99072,
 993

In [6]:
clk104.lmk.update(printDebug=True)

SEL_MODE: CLK_IN_1_MANUAL
Input divider: 8
PLL1 Phase Detector Frequency: 1.25
PLL1N Divider: 128
Expected VCXO Frequency: 160.0
PLL2 Input Frequency: 6.4
PLL2_P: 2
PLL2_N: 192
PLL2 Output Frequency: 2457.6
SYSREF DIVIDER: 320
SYSREF FREQ: 7.68
Output Branch  0 DIV: 10
Output Branch  0 CLK_PD: ENABLED
Output Branch  0 SDCLK_PD: ENABLED
Output Branch  0 DCLK_FMT: LVDS
Output Branch  0 SDCLK_FMT: HSDS_8_MA
Output Branch  0 DCLK_POL: NORMAL
Output Branch  0 SDCLK_POL: NORMAL
Output Branch  0 SDCLK_MUX: SYSREF_OUTPUT
Output Branch  0 DCLK_MUX: DIVIDER_ONLY
Output Branch  0 DCLK_FREQ: 245.76
Output Branch  0 SDCLK_FREQ: 7.68
Output Branch  0 DCLK_ACTIVE: True
Output Branch  0 SDCLK_ACTIVE: True
Output Branch  2 DIV: 10
Output Branch  2 CLK_PD: ENABLED
Output Branch  2 SDCLK_PD: ENABLED
Output Branch  2 DCLK_FMT: POWERDOWN
Output Branch  2 SDCLK_FMT: HSDS_8_MA
Output Branch  2 DCLK_POL: NORMAL
Output Branch  2 SDCLK_POL: NORMAL
Output Branch  2 SDCLK_MUX: SYSREF_OUTPUT
Output Branch  2 DCLK_

In [7]:
clk104.lmk.write_register_dump("clockfile_out.txt")

In [8]:
!diff clockfile.txt clockfile_out.txt

136c136
< R8191	0x1FFF53
\ No newline at end of file
---
> R8191	0x1FFF53


In [9]:
len(bin(0x1fff))-2

13

In [10]:
def parse_register_file(filename, aw=13, dw=8):
    with open(filename, "r") as f:
        lines = f.read().strip().split("\n")

    ret = []

    for line in lines:
        a,b = line.split("\t")
        data = int(b, 16)
        mask = (1 << aw)-1
        data_mask = (1 << dw) - 1

        ret.append((mask & (data >> dw), data & data_mask))

    return ret

In [11]:
prog_file = parse_register_file("clockfile.txt")

In [12]:
print([a[0] for a in prog_file])

[0, 0, 2, 3, 4, 5, 6, 12, 13, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 369, 370, 380, 381, 358, 359, 360, 361, 362, 363, 364, 365, 366, 371, 386, 387, 388, 389, 392, 393, 394, 395, 8189, 8190, 8191]


In [13]:
if 0:
    for addr,data in prog_file:
        # print(f"Explaining register 0x{addr:x} = 0x{data:x}")
        if not addr in register_by_addr:
            print(f"Unhandled register: {hex(addr)}, skipping ...")
            print()
            continue

        register = register_by_addr[addr]
        print(hex(addr))
        register.parse(data)

        for field in register.fields:
            val = field.get()
            if field.name == "CONST":
                continue

            print(f"    {field.name}")
            print(f"        Description: {field.description}")
            print(f"        Value:       {val}")
            if field.valid_type == "enum":
                print(f"        ValDesc:     {field.value_description}")
            print()

In [14]:
lmx_clock_files = [
    "LMX2594_REF-245M76__OUT-0204M80_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-0245M76_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-0491M52_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-1024M00_11052019_D.txt",
    "LMX2594_REF-245M76__OUT-1966M08_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-2211M84_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-2457M60_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-3072M00_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-3932M16_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-4096M00_11052019_D.txt",
    "LMX2594_REF-245M76__OUT-4423M68_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-4915M20_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-5898M24_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-6144M00_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-6881M28_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-7372M80_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-7864M32_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-8192M00_11052019.txt",
    "LMX2594_REF-245M76__OUT-8847M36_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-9830M40_10172019_I.txt"
]

In [15]:
lmx = ipq_pynq_utils.LMX2594(245.76)

In [16]:
for f in lmx_clock_files:
    lmx.init_from_file(f"../ipq_pynq_utils/data/clockFiles/{f}")
    print(f"f_outa = {round(lmx.f_outa, ndigits=2):7.2f}, f_vco = {round(lmx.f_vco, ndigits=2):8.2f}")# , f_outb = {round(lmx.f_outb, ndigits=3)}")

f_outa =  204.80, f_vco =  9830.40
f_outa =  245.76, f_vco =  7864.32
f_outa =  491.52, f_vco =  7864.32
f_outa = 1024.00, f_vco =  8192.00
f_outa = 1966.08, f_vco =  7864.32
f_outa = 2211.84, f_vco =  8847.36
f_outa = 2457.60, f_vco =  9830.40
f_outa = 3072.00, f_vco = 12288.00
f_outa = 3932.16, f_vco =  7864.32
f_outa = 4096.00, f_vco =  8192.00
f_outa = 4423.68, f_vco =  8847.36
f_outa = 4915.20, f_vco =  9830.40
f_outa = 5898.24, f_vco = 11796.48
f_outa = 6144.00, f_vco = 12288.00
f_outa = 6881.28, f_vco = 13762.56
f_outa = 7372.80, f_vco = 14745.60
f_outa = 7864.32, f_vco =  7864.32
f_outa = 8192.00, f_vco =  8192.00
f_outa = 8847.36, f_vco =  8847.36
f_outa = 9830.40, f_vco =  9830.40


In [17]:
num = lmx.get_long_register(lmx.PLL_NUM_31_16, lmx.PLL_NUM_15_0)
den = lmx.get_long_register(lmx.PLL_DEN_31_16, lmx.PLL_DEN_15_0)
pll_n = lmx.get_long_register(lmx.PLL_N_18_16, lmx.PLL_N_15_0)

In [18]:
lmx.f_outa

9830.4

In [19]:
f_pd = 24.576
f_vco = f_pd * (pll_n + num/den)

In [20]:
lmx.OUTA_MUX.get()

VCO

In [21]:
lmx.SYSREF_DIV.get()

1

In [22]:
9879 / (6*4)

411.625